In [129]:
import json
import pandas as pd
import numpy as np
from annoy import AnnoyIndex
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [77]:
# n is the number of json files we wish to read (max is 1000)
n = 5

for i in range(n):
    # create the file_name string in order to read in file
    file_name = 'spotify_million_playlist_dataset/data/mpd.slice.' \
        + str(i*1000) \
        + '-' + str(i*1000+999) + '.json'

    # Uncomment the following line to show progress
    #print(file_name)

    # open the file and store its contents in file_contents
    with open(file_name) as user_file:
        file_contents = user_file.read()

    # we only care about the "playlists" part of this dictionary
    # save the list of playlists in playlist_list
    parsed_json = json.loads(file_contents)
    playlist_list = parsed_json["playlists"]


    # create dataframe if it's first playlist, otherwise append info to existing dataframe
    # the record_path argument tells the json_normalize function how to flatten the data
    # the meta argument tells the json_nomralize function what meta data to keep
    if i == 0:
        data = pd.json_normalize(
            playlist_list,
            record_path="tracks",
            meta=["name", "collaborative", "pid", "num_followers", "num_edits"]
        )
    else:
        data = pd.concat(
            [
                data,
                pd.json_normalize(
                    playlist_list,
                    record_path="tracks",
                    meta=["name", "collaborative", "pid", "num_followers", "num_edits"]
                )
            ],
            ignore_index = True
        )

In [125]:
f=384

t= AnnoyIndex(f, 'angular')

In [126]:
t.load('playlist_vectors.ann')
t.get_n_items()

5000

# Searching example

In [172]:
search = 'tropical'

#generate search vector
#note that i am adding the suffix playlist to everything
v_search = model.encode(search.lower())

In [173]:
#will return the approximate nearest neighbors
#pid's to the search vector
#the order of this list is important: roughly ~ closest -> furthest 
num_neigh= 50
pid_nns = t.get_nns_by_vector(v_search , num_neigh)

In [174]:
df = data.drop_duplicates(subset = ['pid'])

In [175]:
for pid in pid_nns:
    name = list(df.loc[df['pid']== pid,'name'])[0]
    print(name)

Tropical
Tropical
TROPICAL
tropical
Tropicales
Tropics
Cuba
Tropicana
Fiji
Paradise
VACATION
Country summer
country summer 
summer country
Summer Country
Summer country
Summer Country
Summer Country
summer country
summer country
COSTA RICA 
Hawaiian
Hawaii
hawaii
Hawaii 
HAWAII
Island
ISLAND
Around the World
Sky
beach
Beach
beach
Beach
Beach
thai
Disneyland
Florida
Maui
Panda
panda
Panda
rainy days
Rainy Days
Rainy days
Acústico
Acústico 
Country
Country
Country
